In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats 
import matplotlib.pyplot as plt
import hvplot.pandas
from datetime import datetime

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, 
    roc_auc_score, roc_curve, auc,
    # plot_confusion_matrix, plot_roc_curve
)
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

bold_start = "\033[1m"
bold_end = "\033[0m"

Import the dataset

In [2]:
data = pd.read_csv("resources/lending_club_loan_two.csv")

## Data Exploration

In [3]:
data.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.00,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.00,Not Verified,Jan-2015,Fully Paid,vacation,Vacation,26.24,Jun-1990,16.00,0.00,36369.00,41.80,25.00,w,INDIVIDUAL,0.00,0.00,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.00,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.00,Not Verified,Jan-2015,Fully Paid,debt_consolidation,Debt consolidation,22.05,Jul-2004,17.00,0.00,20131.00,53.30,27.00,f,INDIVIDUAL,3.00,0.00,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.00,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.00,Source Verified,Jan-2015,Fully Paid,credit_card,Credit card refinancing,12.79,Aug-2007,13.00,0.00,11987.00,92.20,26.00,f,INDIVIDUAL,0.00,0.00,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.00,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.00,Not Verified,Nov-2014,Fully Paid,credit_card,Credit card refinancing,2.60,Sep-2006,6.00,0.00,5472.00,21.50,13.00,f,INDIVIDUAL,0.00,0.00,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.00,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.00,Verified,Apr-2013,Charged Off,credit_card,Credit Card Refinance,33.95,Mar-1999,13.00,0.00,24584.00,69.80,43.00,f,INDIVIDUAL,1.00,0.00,"679 Luna Roads\r\nGreggshire, VA 11650"


In [4]:
data.describe()


,loan_amnt,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies
count,396030.00,396030.00,396030.00,396030.00,396030.00,396030.00,396030.00,396030.00,395754.00,396030.00,358235.00,395495.00
mean,14113.89,13.64,431.85,74203.18,17.38,11.31,0.18,15844.54,53.79,25.41,1.81,0.12
std,8357.44,4.47,250.73,61637.62,18.02,5.14,0.53,20591.84,24.45,11.89,2.15,0.36
min,500.00,5.32,16.08,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
25%,8000.00,10.49,250.33,45000.00,11.28,8.00,0.00,6025.00,35.80,17.00,0.00,0.00
50%,12000.00,13.33,375.43,64000.00,16.91,10.00,0.00,11181.00,54.80,24.00,1.00,0.00
75%,20000.00,16.49,567.30,90000.00,22.98,14.00,0.00,19620.00,72.90,32.00,3.00,0.00
max,40000.00,30.99,1533.81,8706582.00,9999.00,90.00,86.00,1743266.00,892.30,151.00,34.00,8.00


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amnt             396030 non-null  float64
 1   term                  396030 non-null  object 
 2   int_rate              396030 non-null  float64
 3   installment           396030 non-null  float64
 4   grade                 396030 non-null  object 
 5   sub_grade             396030 non-null  object 
 6   emp_title             373103 non-null  object 
 7   emp_length            377729 non-null  object 
 8   home_ownership        396030 non-null  object 
 9   annual_inc            396030 non-null  float64
 10  verification_status   396030 non-null  object 
 11  issue_d               396030 non-null  object 
 12  loan_status           396030 non-null  object 
 13  purpose               396030 non-null  object 
 14  title                 394274 non-null  object 
 15  

In [6]:
data['loan_status'].value_counts().hvplot.bar(
    title="Loan Status Counts", xlabel='Loan Status', ylabel='Count', 
    width=500, height=350
)

:Bars   [loan_status]   (count)

In [7]:
data['loan_status'].value_counts()


loan_status
Fully Paid     318357
Charged Off     77673
Name: count, dtype: int64

In [8]:
data.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies
count,396030.00,396030.00,396030.00,396030.00,396030.00,396030.00,396030.00,396030.00,395754.00,396030.00,358235.00,395495.00
mean,14113.89,13.64,431.85,74203.18,17.38,11.31,0.18,15844.54,53.79,25.41,1.81,0.12
std,8357.44,4.47,250.73,61637.62,18.02,5.14,0.53,20591.84,24.45,11.89,2.15,0.36
min,500.00,5.32,16.08,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
25%,8000.00,10.49,250.33,45000.00,11.28,8.00,0.00,6025.00,35.80,17.00,0.00,0.00
50%,12000.00,13.33,375.43,64000.00,16.91,10.00,0.00,11181.00,54.80,24.00,1.00,0.00
75%,20000.00,16.49,567.30,90000.00,22.98,14.00,0.00,19620.00,72.90,32.00,3.00,0.00
max,40000.00,30.99,1533.81,8706582.00,9999.00,90.00,86.00,1743266.00,892.30,151.00,34.00,8.00


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amnt             396030 non-null  float64
 1   term                  396030 non-null  object 
 2   int_rate              396030 non-null  float64
 3   installment           396030 non-null  float64
 4   grade                 396030 non-null  object 
 5   sub_grade             396030 non-null  object 
 6   emp_title             373103 non-null  object 
 7   emp_length            377729 non-null  object 
 8   home_ownership        396030 non-null  object 
 9   annual_inc            396030 non-null  float64
 10  verification_status   396030 non-null  object 
 11  issue_d               396030 non-null  object 
 12  loan_status           396030 non-null  object 
 13  purpose               396030 non-null  object 
 14  title                 394274 non-null  object 
 15  

In [10]:
data.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.00,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.00,Not Verified,Jan-2015,Fully Paid,vacation,Vacation,26.24,Jun-1990,16.00,0.00,36369.00,41.80,25.00,w,INDIVIDUAL,0.00,0.00,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.00,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.00,Not Verified,Jan-2015,Fully Paid,debt_consolidation,Debt consolidation,22.05,Jul-2004,17.00,0.00,20131.00,53.30,27.00,f,INDIVIDUAL,3.00,0.00,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.00,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.00,Source Verified,Jan-2015,Fully Paid,credit_card,Credit card refinancing,12.79,Aug-2007,13.00,0.00,11987.00,92.20,26.00,f,INDIVIDUAL,0.00,0.00,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.00,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.00,Not Verified,Nov-2014,Fully Paid,credit_card,Credit card refinancing,2.60,Sep-2006,6.00,0.00,5472.00,21.50,13.00,f,INDIVIDUAL,0.00,0.00,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.00,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.00,Verified,Apr-2013,Charged Off,credit_card,Credit Card Refinance,33.95,Mar-1999,13.00,0.00,24584.00,69.80,43.00,f,INDIVIDUAL,1.00,0.00,"679 Luna Roads\r\nGreggshire, VA 11650"


## TODO: Remove this blocks below, these blocks below are supposed to be in Data Exploration: Talk to Andrew about this

In [11]:
def pub_rec(number):
    if number == 0.0:
        return 0
    else:
        return 1
    
def mort_acc(number):
    if number == 0.0:
        return 0
    elif number >= 1.0:
        return 1
    else:
        return number
    
def pub_rec_bankruptcies(number):
    if number == 0.0:
        return 0
    elif number >= 1.0:
        return 1
    else:
        return number

In [12]:
data['pub_rec'] = data.pub_rec.apply(pub_rec)
data['mort_acc'] = data.mort_acc.apply(mort_acc)
data['pub_rec_bankruptcies'] = data.pub_rec_bankruptcies.apply(pub_rec_bankruptcies)

data['loan_status'] = data.loan_status.map({'Fully Paid':1, 'Charged Off':0})

## 2. Data Preprocessing

The goal here is to do the following:

* Remove or impute missing data
* Remove repeating and duplicate features
* Convert categorical string features into numerical formats

In [13]:
# The length of the data
print(f"The Length of the data: {data.shape}")

The Length of the data: (396030, 27)


### 2.2 Converting categorical string features into numerical formats

In [14]:
data.term.unique()

array([' 36 months', ' 60 months'], dtype=object)

In [15]:
term_values = {' 36 months': 36, ' 60 months': 60}
data['term'] = data.term.map(term_values)

In [16]:
data.term.unique()

array([36, 60], dtype=int64)

In [17]:
data.drop('grade', axis=1, inplace=True)

In [18]:
dummies = ['sub_grade', 'verification_status', 'purpose', 'initial_list_status', 
           'application_type', 'home_ownership']
data = pd.get_dummies(data, columns=dummies, drop_first=True)

address

We are going to feature engineer a zip code column from the address in the data set. Create a column called 'zip_code' that extracts the zip code from the address column.

In [19]:
data.address.head()

0       0174 Michelle Gateway\r\nMendozaberg, OK 22690
1    1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113
2    87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113
3              823 Reid Ford\r\nDelacruzside, MA 00813
4               679 Luna Roads\r\nGreggshire, VA 11650
Name: address, dtype: object

In [20]:
data['zip_code'] = data.address.apply(lambda x: x[-5:])

In [21]:
data.zip_code.value_counts()

zip_code
70466    56985
30723    56546
22690    56527
48052    55917
00813    45824
29597    45471
05113    45402
11650    11226
93700    11151
86630    10981
Name: count, dtype: int64

In [22]:
# data = pd.get_dummies(data, columns=['zip_code'], drop_first=True)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the zip_code column
data['zip_code_encoded'] = label_encoder.fit_transform(data['zip_code'])

data.drop('zip_code', axis=1, inplace =True)

In [23]:
data.drop('address', axis=1, inplace=True)

issue_d

This would be data leakage, we wouldn't know beforehand whether or not a loan would be issued when using our model, so in theory we wouldn't have an issue_date, drop this feature.

In [24]:
data.drop('issue_d', axis=1, inplace=True)

In [25]:
data.earliest_cr_line.dtype

dtype('O')

In [26]:
data['earliest_cr_line'] = data['earliest_cr_line'].apply(lambda x: datetime.strptime(x, "%b-%Y").year)

In [27]:
data.earliest_cr_line.nunique()

65

In [28]:
data.earliest_cr_line.value_counts()

earliest_cr_line
2000    29366
2001    29083
1999    26491
2002    25901
2003    23657
        ...  
1951        3
1950        3
1953        2
1944        1
1948        1
Name: count, Length: 65, dtype: int64

### 2.1 Remove or impute missing data

In [29]:
for column in data.columns:
    if data[column].isna().sum() != 0:
        missing = data[column].isna().sum()
        portion = (missing / data[column].shape[0]) * 100
        print(f"{bold_start}'{column}'{bold_end}: number of missing values: {bold_start}'{missing}'{bold_end} ==> {bold_start}'{portion:.3f}%'{bold_end}")

'emp_title': number of missing values: '22927' ==> '5.789%'
'emp_length': number of missing values: '18301' ==> '4.621%'
'title': number of missing values: '1756' ==> '0.443%'
'revol_util': number of missing values: '276' ==> '0.070%'
'mort_acc': number of missing values: '37795' ==> '9.543%'
'pub_rec_bankruptcies': number of missing values: '535' ==> '0.135%'


In [30]:
data['emp_title'].nunique()

173105

The **emp_title** has too many unique values to encode and dataset will not be sufficent if used with it. So, the best thing to do is drop it.

In [31]:
data.drop('emp_title', axis=1, inplace=True)

Now, let's look at **emp_length**

In [32]:
data["emp_length"].unique()

array(['10+ years', '4 years', '< 1 year', '6 years', '9 years',
       '2 years', '3 years', '8 years', '7 years', '5 years', '1 year',
       nan], dtype=object)

In [33]:
for year in data["emp_length"].unique():
    print(f"{bold_start}{year}{bold_end} years in the positions:")
    print(f"{data[data.emp_length == year].loan_status.value_counts(normalize=True)}")
    print("\n=============================================\n")

10+ years years in the positions:
loan_status
1   0.82
0   0.18
Name: proportion, dtype: float64


4 years years in the positions:
loan_status
1   0.81
0   0.19
Name: proportion, dtype: float64


< 1 year years in the positions:
loan_status
1   0.79
0   0.21
Name: proportion, dtype: float64


6 years years in the positions:
loan_status
1   0.81
0   0.19
Name: proportion, dtype: float64


9 years years in the positions:
loan_status
1   0.80
0   0.20
Name: proportion, dtype: float64


2 years years in the positions:
loan_status
1   0.81
0   0.19
Name: proportion, dtype: float64


3 years years in the positions:
loan_status
1   0.80
0   0.20
Name: proportion, dtype: float64


8 years years in the positions:
loan_status
1   0.80
0   0.20
Name: proportion, dtype: float64


7 years years in the positions:
loan_status
1   0.81
0   0.19
Name: proportion, dtype: float64


5 years years in the positions:
loan_status
1   0.81
0   0.19
Name: proportion, dtype: float64


1 year years in the positio

**Charge offs** seem to be extremely similar across all **emp_length**. So, we are going to drop it.

In [34]:
data.drop('emp_length', axis=1, inplace=True)

Now, let's have a look at **title**.

In [35]:
data.title.value_counts().head()

title
Debt consolidation         152472
Credit card refinancing     51487
Home improvement            15264
Other                       12930
Debt Consolidation          11608
Name: count, dtype: int64

In [36]:
data.title.unique().shape

(48817,)

In [37]:
data.drop("title", axis=1, inplace=True)

Let's have a look at **mort_acc**.

In [38]:
data.mort_acc.value_counts()

mort_acc
1.00    218458
0.00    139777
Name: count, dtype: int64

In [39]:
data.mort_acc.isna().sum()

37795

**mort_acc** should be imputed using either mean or prediction model, for this purpose we will do using **Random Forest**

In [40]:
# Split the data into two parts: one with missing values and one without
missing_data = data[data['mort_acc'].isnull()]
complete_data = data.dropna(subset=['mort_acc'])

# Select features and target variable
X = complete_data.drop(columns=['mort_acc'])
y = complete_data['mort_acc']

# Split the complete data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict missing values
imputed_values = model.predict(missing_data.drop(columns=['mort_acc']))

# Impute missing values in the mort_acc column
data.loc[data['mort_acc'].isnull(), 'mort_acc'] = imputed_values

In [41]:
data.mort_acc.value_counts()

mort_acc
1.00    237653
0.00    158377
Name: count, dtype: int64

revol_util & pub_rec_bankruptcies

These two features have missing data points, but they account for less than 0.5% of the total data. So we are going to remove the rows that are missing those values in those columns with dropna().

In [42]:
for column in data.columns:
    if data[column].isna().sum() != 0:
        missing = data[column].isna().sum()
        portion = (missing / data.shape[0]) * 100
        print(f"'{column}': number of missing values '{missing}' ==> '{portion:.3f}%'")

'revol_util': number of missing values '276' ==> '0.070%'
'pub_rec_bankruptcies': number of missing values '535' ==> '0.135%'


In [43]:
data.dropna(inplace=True)

In [44]:
data.shape

(395219, 73)

In [45]:
data.loan_status.value_counts()

loan_status
1    317696
0     77523
Name: count, dtype: int64

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395219 entries, 0 to 396029
Data columns (total 73 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   loan_amnt                            395219 non-null  float64
 1   term                                 395219 non-null  int64  
 2   int_rate                             395219 non-null  float64
 3   installment                          395219 non-null  float64
 4   annual_inc                           395219 non-null  float64
 5   loan_status                          395219 non-null  int64  
 6   dti                                  395219 non-null  float64
 7   earliest_cr_line                     395219 non-null  int64  
 8   open_acc                             395219 non-null  float64
 9   pub_rec                              395219 non-null  int64  
 10  revol_bal                            395219 non-null  float64
 11  revol_util        

## 3. Preparing the Test data

### 3.1 Checking distribution of data

In [47]:
# Define a threshold for Z-score
threshold = 3

# Create a copy of the data to avoid modifying the original DataFrame
data_clean = data.copy()

# Dictionary to store counts of outliers for each column
outliers_count_dict = {}

# Loop through each numerical column in the DataFrame
for column in data_clean.select_dtypes(include=[np.number]).columns:
    # Calculate Z-scores
    z_scores = (data_clean[column] - data_clean[column].mean()) / data_clean[column].std()
    # Identify outliers
    outliers = np.abs(z_scores) > threshold
    # Store the count of previous outliers
    outliers_count_dict[column] = outliers.sum()
    # Calculate the mean value of the column
    mean_value = data_clean[column].mean()
    # If the column is of type int64, cast the mean value to int
    if pd.api.types.is_integer_dtype(data_clean[column]):
        mean_value = int(mean_value)
    # Replace outliers with the mean value of the column
    data_clean.loc[outliers, column] = mean_value

# Print the shape of the original and cleaned data
print(f"Original data shape: {data.shape}")
print(f"Cleaned data shape: {data_clean.shape}")

# Print count of previous outliers
print("\nCount of previous outliers in each column:")
for column, count in outliers_count_dict.items():
    print(f"Column: {column} - Outliers: {count}")

# Optional: Check if there are any remaining outliers
remaining_outliers = ((data_clean - data_clean.mean()) / data_clean.std()).abs() > threshold
print("\nRemaining outliers in each column after cleaning:")
print(remaining_outliers.sum())

Original data shape: (395219, 73)
Cleaned data shape: (395219, 73)

Count of previous outliers in each column:
Column: loan_amnt - Outliers: 184
Column: term - Outliers: 0
Column: int_rate - Outliers: 754
Column: installment - Outliers: 5042
Column: annual_inc - Outliers: 3190
Column: loan_status - Outliers: 0
Column: dti - Outliers: 12
Column: earliest_cr_line - Outliers: 4993
Column: open_acc - Outliers: 4873
Column: pub_rec - Outliers: 0
Column: revol_bal - Outliers: 4771
Column: revol_util - Outliers: 16
Column: total_acc - Outliers: 3396
Column: mort_acc - Outliers: 0
Column: pub_rec_bankruptcies - Outliers: 0
Column: zip_code_encoded - Outliers: 0

Remaining outliers in each column after cleaning:
loan_amnt                   0
term                        0
int_rate                   51
installment              2700
annual_inc               6888
                        ...  
home_ownership_NONE        29
home_ownership_OTHER      110
home_ownership_OWN      37660
home_ownership_RE

In [48]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from collections import Counter

# Assuming data_clean is your DataFrame and 'target' is the target variable
X = data_clean.drop(columns=['target'])
y = data_clean['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(f"Original dataset shape: {Counter(y_train)}")
print(f"Resampled dataset shape: {Counter(y_train_res)}")

ModuleNotFoundError: No module named 'imblearn'